In [1]:
import sys
import nltk
import sklearn
import pandas
import numpy

import pandas as pd
import numpy as np
df = pd.read_csv('Negf.csv')
print (df.head(3))

  App_name   Category                               User_Review Sentiments  \
0  brainly  Education  Not working properly plz fix the problem   Negative   
1  brainly  Education         App is not good, stuck in between   Negative   
2  brainly  Education                      not working properly   Negative   

  Review_Type  
0          AC  
1          AC  
2          AC  


In [2]:
classes = df['Review_Type']
print(classes.value_counts())

PI    2403
BR    2201
AC    1129
FI     937
CI     873
VU     777
AI     363
FR     223
UI     206
Name: Review_Type, dtype: int64


In [3]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform (classes)
print(classes[:8])
print(Y[:8])
priority = df['Review_Type']
print(priority[:10])


0    AC
1    AC
2    AC
3    AC
4    AC
5    AC
6    AC
7    AC
Name: Review_Type, dtype: object
[0 0 0 0 0 0 0 0]
0    AC
1    AC
2    AC
3    AC
4    AC
5    AC
6    AC
7    AC
8    AC
9    AC
Name: Review_Type, dtype: object


In [4]:
# Categorical boolean mask
categorical_feature_mask = df.dtypes == object

In [5]:
categorical_feature_mask


App_name       True
Category       True
User_Review    True
Sentiments     True
Review_Type    True
dtype: bool

In [6]:
# filter categorical columns using mask and turn it into a list
categorical_cols = df.columns[categorical_feature_mask].tolist()
categorical_cols

['App_name', 'Category', 'User_Review', 'Sentiments', 'Review_Type']

In [7]:
pri_col = []
for i in categorical_cols:
    if i != 'Summary':
        pri_col.append(i)
print(pri_col)

['App_name', 'Category', 'User_Review', 'Sentiments', 'Review_Type']


In [8]:
le = LabelEncoder()
df[pri_col] = df[pri_col].apply(lambda col: le.fit_transform(col))
df[pri_col].head(10)


,App_name,Category,User_Review,Sentiments,Review_Type
0,19,0,2755,0,0
1,19,0,261,0,0
2,19,0,5335,0,0
3,19,0,4958,0,0
4,19,0,5478,0,0
5,19,0,5391,0,0
6,19,0,4964,0,0
7,19,0,2308,0,0
8,19,0,5335,0,0
9,19,0,12,0,0


In [9]:
df.Review_Type.value_counts()

6    2403
2    2201
0    1129
4     937
3     873
8     777
1     363
5     223
7     206
Name: Review_Type, dtype: int64

In [10]:
from sklearn.utils import resample
df_minority0 = df[df.Review_Type==0]
df_minority1 = df[df.Review_Type==4]
df_minority2 = df[df.Review_Type==3]
df_minority3 = df[df.Review_Type==8]
df_minority4 = df[df.Review_Type==1]
df_minority5 = df[df.Review_Type==5]
df_minority6 = df[df.Review_Type==7]


In [11]:
df_minority_upsampled0 = resample(df_minority0, 
                                 replace=True,     # sample with replacement
                                 n_samples=950)    # to match majority class
df_minority_upsampled1 = resample(df_minority4, 
                                 replace=True,     # sample with replacement
                                 n_samples=1200)    # to match majority class
df_minority_upsampled2 = resample(df_minority3, 
                                 replace=True,     # sample with replacement
                                 n_samples=1300)    # to match majority class
df_minority_upsampled3 = resample(df_minority1, 
                                 replace=True,     # sample with replacement
                                 n_samples=1800)
df_minority_upsampled4 = resample(df_minority5, 
                                 replace=True,     # sample with replacement
                                 n_samples=1800)
df_minority_upsampled5 = resample(df_minority6, 
                                 replace=True,     # sample with replacement
                                 n_samples=1800)

In [12]:
df = pd.concat([df, df_minority_upsampled0, df_minority_upsampled1, df_minority_upsampled2, df_minority_upsampled3, df_minority_upsampled4, df_minority_upsampled5])


In [13]:
df.Review_Type.value_counts()


4    2737
6    2403
2    2201
0    2079
8    2077
5    2023
7    2006
1    1563
3     873
Name: Review_Type, dtype: int64

In [14]:
df_minority_upsampled6 = resample(df_minority2, 
                                 replace=True,     # sample with replacement
                                 n_samples=1800)

In [15]:
df = pd.concat([df, df_minority_upsampled6])


In [16]:
df.Review_Type.value_counts()


4    2737
3    2673
6    2403
2    2201
0    2079
8    2077
5    2023
7    2006
1    1563
Name: Review_Type, dtype: int64

In [17]:
X  = df[['User_Review']]


In [18]:
print(X.shape)


(19762, 1)


In [19]:
Y = df['Review_Type']


In [20]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


In [21]:
print(Y.value_counts())


4    2737
3    2673
6    2403
2    2201
0    2079
8    2077
5    2023
7    2006
1    1563
Name: Review_Type, dtype: int64


In [ ]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X_train, y_train)

In [ ]:
predictions = text_classifier.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)


In [ ]:
print(confusion_matrix(y_test,y_pred))  
print(metrics.accuracy_score(y_test, y_pred))
print(classification_report(y_test,y_pred))  


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(solver='liblinear', random_state=0)
#fit the model
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred_test))  
print(metrics.accuracy_score(y_test, y_pred_test))
print(classification_report(y_test,y_pred_test))

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(criterion = 'entropy').fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 
print("Accuracy:",metrics.accuracy_score(y_test, dtree_predictions))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,dtree_predictions))  
print(classification_report(y_test,dtree_predictions))  


In [ ]:
from sklearn import svm
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel
#Train the model using the training sets
clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)



In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
